In [333]:
import os
import json
import pandas as pd
import traceback

In [334]:
from langchain.chat_models import ChatOpenAI

IGNORE

START

In [327]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [329]:
from dotenv import load_dotenv, find_dotenv
import os

# Explicitly find the .env file
dotenv_path = find_dotenv()
if dotenv_path == "":
    print("Could not find .env file")
else:
    print(f".env file found at: {dotenv_path}")

# Load environment variables from .env file
load_dotenv(dotenv_path)

# Retrieve the environment variable
KEY = os.getenv('OPENAI_API_KEY')

# Print the key to verify
print(f'API Key: {KEY}')


.env file found at: c:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\.env
API Key: setx OPENAI_API_KEY your-api-key-here


END 

In [361]:
import os

# Set the OpenAI API key directly
os.environ['OPENAI_API_KEY'] = 'sk-proj-yDirk9Kal6pNFSoTU48QT3BlbkFJo9A1XnmyWUf6L35WJ8LK'

# Now retrieve the environment variable
KEY = os.getenv('OPENAI_API_KEY')

# Print the key to verify
print(f'API Key: {KEY}')


API Key: sk-proj-yDirk9Kal6pNFSoTU48QT3BlbkFJo9A1XnmyWUf6L35WJ8LK


In [364]:
llm=ChatOpenAI(openai_api_key
               =KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [365]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014FB6891210>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014FB6893C40>, temperature=0.5, openai_api_key='sk-proj-yDirk9Kal6pNFSoTU48QT3BlbkFJo9A1XnmyWUf6L35WJ8LK', openai_proxy='')

In [366]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [367]:
with open(r"C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\Response.json", "r") as f:
    RESPONSE_JSON=json.load(f)

In [368]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [369]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [370]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
)

In [371]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [372]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [373]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    )

In [374]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [375]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    output_variables=["quiz", "review"], 
    verbose=True,
    )

In [376]:
PATH = r"C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\data.txt"

In [377]:
print(PATH)

C:\Users\Aman\Desktop\genai MCQ Project\MCQ-Generator-Genai-Project\data.txt


In [378]:
with open(PATH, "r") as file:
    TEXT=file.read()    

In [379]:
TEXT

'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: \n\n\nSystems, architecture and infrastructure for Gen AI\nFoundation models and their applications\nMulti-modal Gen AI \nRetrieval Augmented Generated\nGen AI based plugins and agents\nLearning from human preferences \nEvaluation of f

In [380]:
##MCQ ko generate krwana hai

In [381]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [382]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [383]:
NUMBER=5 
SUBJECT="AI"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON

In [384]:
response_json_str=json.dumps(RESPONSE_JSON)

In [385]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [386]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": response_json_str
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: 


Systems, architecture and infrastructure for Gen AI
Foundation models and their applications
Multi-modal Gen AI 
Retrieval Au

In [387]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1256
Prompt Tokens:886
Completion Tokens:370
Total Cost:0.002069


In [389]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to: \n\n\nSystems, architecture and infrastructure for Gen AI\nFoundation models and their applications\nMulti-modal Gen AI \nRetrieval Augmented Generated\nGen AI based plugins and agents\nLearning from human preferences \nEvalua

In [390]:
quiz=response.get("quiz")

In [391]:
quiz= json.loads(quiz)

In [392]:
quiz

{'1': {'mcq': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Building robots',
   'b': 'Creating realistic text, images, and video',
   'c': 'Developing mobile apps',
   'd': 'Designing websites'},
  'correct': 'b'},
 '2': {'mcq': 'What are some of the topics of interest for the workshop on Generative AI?',
  'options': {'a': 'Cooking recipes',
   'b': 'Systems, architecture, and infrastructure for Gen AI',
   'c': 'Sports statistics',
   'd': 'Fashion design'},
  'correct': 'b'},
 '3': {'mcq': 'What is one of the applications enabled by Generative AI?',
  'options': {'a': 'Weather forecasting',
   'b': 'Creating music',
   'c': 'Building bridges',
   'd': 'Farming'},
  'correct': 'b'},
 '4': {'mcq': 'What is a concern related to Generative AI technology?',
  'options': {'a': 'Efficiency',
   'b': 'Fairness, transparency, and accountability',
   'c': 'Speed',
   'd': 'Cost'},
  'correct': 'b'},
 '5': {'mcq': 'What is one of the topics covered 

In [393]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join( 
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [394]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Building robots | b: Creating realistic text, images, and video | c: Developing mobile apps | d: Designing websites',
  'Correct': 'b'},
 {'MCQ': 'What are some of the topics of interest for the workshop on Generative AI?',
  'Choices': 'a: Cooking recipes | b: Systems, architecture, and infrastructure for Gen AI | c: Sports statistics | d: Fashion design',
  'Correct': 'b'},
 {'MCQ': 'What is one of the applications enabled by Generative AI?',
  'Choices': 'a: Weather forecasting | b: Creating music | c: Building bridges | d: Farming',
  'Correct': 'b'},
 {'MCQ': 'What is a concern related to Generative AI technology?',
  'Choices': 'a: Efficiency | b: Fairness, transparency, and accountability | c: Speed | d: Cost',
  'Correct': 'b'},
 {'MCQ': 'What is one of the topics covered under Responsible Gen AI?',
  'Choices': 'a: Cooking techniques | b: Large language model performance benchmarkin

In [395]:
df=pd.DataFrame(quiz_table_data)

In [396]:
df

,MCQ,Choices,Correct
0,What is the focus of the workshop on Generativ...,a: Building robots | b: Creating realistic tex...,b
1,What are some of the topics of interest for th...,"a: Cooking recipes | b: Systems, architecture,...",b
2,What is one of the applications enabled by Gen...,a: Weather forecasting | b: Creating music | c...,b
3,What is a concern related to Generative AI tec...,"a: Efficiency | b: Fairness, transparency, and...",b
4,What is one of the topics covered under Respon...,a: Cooking techniques | b: Large language mode...,b


In [220]:
df.to_csv("genai.csv", index=False)